In [2]:
import importlib
import numpy as np
import matplotlib.pyplot as plt
import PyMieScatt as ps
import sphere_difraction as sd
import scipy.constants as const
import ploting_functions as ploting

importlib.reload(sd)
importlib.reload(ploting)

c = const.speed_of_light

## Tasks descriptions

In [ ]:
def SingeLayered():
    lmbd = c / (0.3 * 1e9)  # wave length
    k = 2 * np.pi / lmbd # wave number

    eps = np.array([2.56-0.102j, # permittivities of layers
                    1.0] ) # permittivity of outer space (air)
    
    r = np.array([0.5]) # concentric spheres radiuses

    return r, eps, lmbd

In [24]:
def TwoLayered():
    lmbd = 1 # wave length
    k = 2 * np.pi / lmbd # wave number

    eps = np.array([1, 2.56 + 0.102j,  # permittivities of layers
                    1.0]) # permittivity of vacuum (outer space)

    r = np.array([0.5, 0.55]) # concentric spheres radiuses

    return r, eps, lmbd

In [25]:
def ThreeLayered():
     lmbd = 2 * np.pi / 1000.0 # wave length
     k = 2 * np.pi / lmbd # wave number

     eps = np.array([1, 1.1 + 0.01j, 2.5, # permittivities of layers
                     1.0] ) # permittivity of outer space (air) 

     r = np.array([0.01, 
          0.01 + np.pi/k, 
          0.01 + np.pi/k + 0.5 /k]) # concentric spheres radiuses
     
     return r, eps, lmbd

In [26]:
def FourLayered():
    lmbd = 2 * np.pi / 1000.0 # wave length
    k = 2 * np.pi / lmbd # wave number

    eps = np.array([1, 2.5, 0.25, # permittivities of layers
                    8.85e-12]) # permittivity of vacuum (outer space)

    r = np.array([0.9/4,
        0.9/3,
        0.9/2,
        0.9]) # concentric spheres radiuses
    
    return r, eps, lmbd


In [27]:
def SilverDot():
    lmbd = 3e2 # wave length

    eps = np.array([(1.5258 + 1.8878j)**2, # permittivities of layers
                    1.0]) # permittivity of air (outer space)

    r = np.array([5]) # concentric spheres radiuses

    return r, eps, lmbd

## Plots

In [ ]:
importlib.reload(sd)
importlib.reload(ploting)
lmbd =  c / (1.0 * 1e9) # wave length
k = 2 * np.pi / lmbd # wave number

eps = np.array([2.56, # permittivities of layers
                1.0] ) # permittivity of outer space (air)

r = np.array([0.5]) # concentric spheres radiuses
conducting_core = False

# E_theta, _ =  sd.calculate_electric_field_far(r, eps, lmbd, conducting_core)
# ploting.plot_field_scaterring(E_theta)

D_e, D_m = sd.calculate_coefficients(k, eps, r, conducting_core)
S_th, S_ph = sd.calculate_S(D_e, D_m)

#ploting.plot_radar_cross_section(S_th, k, [-25, 25,10])
#ploting.plot_radar_cross_section(S_ph, k, [-25, 25,10])
ploting.plot_radar_cross_section(S_th, S_ph, k, [-25, 25,10])

## 2D Distribution

In [ ]:
importlib.reload(sd)

r, eps, lmbd = ThreeLayered()
limits = [-2 * r[-1], 8 * r[-1], -5 * r[-1], 5 * r[-1]]

vE_r, vE_theta, vE_phi = sd.calculate_electric_field_close_vectorized(r, eps, lmbd, limits, True)
E = abs(np.sqrt(vE_r**2 + vE_theta**2 + vE_phi**2))

In [ ]:
ploting.plot_field_distribution(E.T, limits, r, eps, lmbd)

### PyMieScatt test

In [ ]:
r, eps, lmbd = SilverDot()
# параметры частицы
wavelength = lmbd     # nm
diameter = r          # nm
m = eps[:-1]          # комплексный показатель преломления
medium_index = eps[-1]       # воздух

MaxAngle = 360
angles = np.linspace(0, MaxAngle, MaxAngle * 2)
theta, SL, SR, SU = ps.ScatteringFunction(m, wavelength, diameter, maxAngle=MaxAngle, nMedium=medium_index)
#theta, SL, SR, SU = ps.CoreShellScatteringFunction(m[0] * 1e2, m[1], wavelength, diameter[0], diameter[1], maxAngle=MaxAngle, nMedium=medium_index, normed=False)

plt.figure(figsize=(7,6))
ax = plt.subplot(111, polar=True)

#ax.plot(theta, SL, label='S⊥')
ax.plot(theta, SR, label='S||')
ax.set_title("Диаграмма рассеяния Mie (полярные координаты)")
ax.legend(loc="upper right")

plt.show()

## Widgets setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets
from IPython.display import display

def plot_function(RE_eps1=1.0, IM_eps1=0.0, RE_eps2=2.5, IM_eps2=0.0, lmbd = 2 * np.pi / 1000.0, d1 = np.pi/1000, d2 = (np.pi + 0.5)/1000):
    r, eps, _ = ThreeLayered()
    eps[1] = complex(RE_eps1, IM_eps1)
    eps[2] = complex(RE_eps2, IM_eps2)

    #r[1] = r[0] + d1
    #r[2] = r[1] + d2


    limits = [-3 * r[-1], 5 * r[-1], -4 * r[-1], 4 * r[-1]]
    vE_r, vE_theta, vE_phi = sd.calculate_electric_field_close_vectorized(r, eps, lmbd, limits)
    E = abs(np.sqrt(vE_r**2 + vE_theta**2 + vE_phi**2))

    plot_field_distribution(E.T, limits, r, eps, lmbd)


slider_RE_eps1 = widgets.FloatSlider(value=1.0, min=0, max=5, step=0.1)
slider_IM_eps1 = widgets.FloatSlider(value=0.0, min=0, max=0.1, step=0.01)
slider_RE_eps2 = widgets.FloatSlider(value=2.5, min=0, max=5, step=0.1)
slider_IM_eps2 = widgets.FloatSlider(value=0.0, min=0, max=0.1, step=0.01)
#slider_Lambda = widgets.FloatSlider(value=2 * np.pi / 1000.0 , min=0.001, max=0.1, step=0.001)

slider_d1 = widgets.FloatLogSlider(value=0.01, min=0.0, max=0.01, step=0.001 )
slider_d2 = widgets.FloatLogSlider(value=0.01, min=0.0, max=0.01, step=0.001 )

label_EPS = widgets.HTML(value="<b>Layers permittivities</b>")
label_Ds = widgets.HTML(value="<b>Layers thickness</b>")

label_RE_EPS1 = widgets.HTML(value="<b>Re eps1</b>")
label_IM_EPS1 = widgets.HTML(value="<b>Im eps1</b>")
label_RE_EPS2 = widgets.HTML(value="<b>Re eps2</b>")
label_IM_EPS2 = widgets.HTML(value="<b>Im eps2</b>")

column1 = widgets.VBox([label_RE_EPS1, label_IM_EPS1, label_RE_EPS2, label_IM_EPS2])
column2 = widgets.VBox([slider_RE_eps1, slider_IM_eps1, slider_RE_eps2, slider_IM_eps2])#, slider_Lambda])
#column3 = widgets.VBox([label_Ds, slider_d1, slider_d2])


controls = widgets.HBox([column1, column2])#, column3])

# Display the layout
display(controls)

out = widgets.interactive_output(plot_function, {
    'RE_eps1': slider_RE_eps1,
    'IM_eps1': slider_IM_eps1,
    'RE_eps2': slider_RE_eps2,
    'IM_eps2': slider_IM_eps2,
    #'lmbd' : slider_Lambda,
    #'d1': slider_d1,
    #'d2': slider_d2,
})

display(out)

# Mass diploma examples image production

In [ ]:
importlib.reload(sd)
lambdas = [c / (0.3 * 1e9) , c / (1.0 * 1e9) , c / (2.0 * 1e9) ]
lambdas_rounded = [1.0, 0.3, 0.15]

y_params = [[-10, 15, 5], [-20, 30, 10], [-20, 30, 10]]

# Simple conducting sphere
r = np.array([0.5]) # concentric spheres radiuses
conducting_core = True

# Lossless dielectric sphere
eps = np.array([2.56, 1.0] )
i = 0

for lmbd in lambdas:
    k = 2 * np.pi / lmbd 
    frequency = 0.3 / lambdas_rounded[i]
    save_path_VV = f"../Theory/images/Conducting_sphere/Conducting_sphere_{frequency}GHz_VV.png"
    save_path_HH = f"../Theory/images/Conducting_sphere/Conducting_sphere_{frequency}GHz_HH.png"

    D_e, D_m = sd.calculate_coefficients(k, eps, r, conducting_core)
    S_th, S_ph = sd.calculate_S(D_e, D_m)
    ploting.plot_radar_cross_section(S_th, k, y_params[i], save_path_VV)
    ploting.plot_radar_cross_section(S_ph, k, y_params[i], save_path_HH)
    i += 1


y_params = [[-30,30,10], [-30,30,10], [-40,30,10]]

# Simple sphere
r = np.array([0.5]) # concentric spheres radiuses
conducting_core = False

# Lossless dielectric sphere
eps = np.array([2.56, 1.0] )
i = 0

for lmbd in lambdas:
    k = 2 * np.pi / lmbd 
    frequency = 0.3 / lambdas_rounded[i]
    save_path_VV = f"../Theory/images/Lossless_dielectric/Lossless_dielectric_{frequency}GHz_VV.png"
    save_path_HH = f"../Theory/images/Lossless_dielectric/Lossless_dielectric_{frequency}GHz_HH.png"

    D_e, D_m = sd.calculate_coefficients(k, eps, r, conducting_core)
    S_th, S_ph = sd.calculate_S(D_e, D_m)
    ploting.plot_radar_cross_section(S_th, k, y_params[i], save_path_VV)
    ploting.plot_radar_cross_section(S_ph, k, y_params[i], save_path_HH)
    i += 1

# Lossy dielectric sphere
eps = np.array([2.56 + 0.102j, 1.0] )
i = 0

for lmbd in lambdas:
    k = 2 * np.pi / lmbd 
    frequency = 0.3 / lambdas_rounded[i]
    save_path_VV = f"../Theory/images/Lossy_dielectric/Lossy_dielectric_{frequency}GHz_VV.png"
    save_path_HH = f"../Theory/images/Lossy_dielectric/Lossy_dielectric_{frequency}GHz_HH.png"

    D_e, D_m = sd.calculate_coefficients(k, eps, r, conducting_core)
    S_th, S_ph = sd.calculate_S(D_e, D_m)
    ploting.plot_radar_cross_section(S_th, k, y_params[i], save_path_VV)
    ploting.plot_radar_cross_section(S_ph, k, y_params[i], save_path_HH)
    i += 1



# Two layered
y_params = [[-20,20,10], [-10,30,10], [-10,30,10]]
r = np.array([0.5, 0.55]) # concentric spheres radiuses
conducting_core = True

# Lossless coated sphere
eps = np.array([1.0, 2.56, 1.0] )
i = 0

for lmbd in lambdas:
    k = 2 * np.pi / lmbd 
    frequency = 0.3 / lambdas_rounded[i]
    save_path_VV = f"../Theory/images/Lossless_coated/Lossless_coated_{frequency}GHz_VV.png"
    save_path_HH = f"../Theory/images/Lossless_coated/Lossless_coated_{frequency}GHz_HH.png"

    D_e, D_m = sd.calculate_coefficients(k, eps, r, conducting_core)
    S_th, S_ph = sd.calculate_S(D_e, D_m)
    ploting.plot_radar_cross_section(S_th, k, y_params[i], save_path_VV)
    ploting.plot_radar_cross_section(S_ph, k, y_params[i], save_path_HH)
    i += 1

# Lossy coated sphere
eps = np.array([1.0, 2.56 + 0.102j, 1.0] )
i = 0

for lmbd in lambdas:
    k = 2 * np.pi / lmbd 
    frequency = 0.3 / lambdas_rounded[i]
    save_path_VV = f"../Theory/images/Lossy_coated/Lossy_coated_{frequency}GHz_VV.png"
    save_path_HH = f"../Theory/images/Lossy_coated/Lossy_coated_{frequency}GHz_HH.png"

    D_e, D_m = sd.calculate_coefficients(k, eps, r, conducting_core)
    S_th, S_ph = sd.calculate_S(D_e, D_m)
    ploting.plot_radar_cross_section(S_th, k, y_params[i], save_path_VV)
    ploting.plot_radar_cross_section(S_ph, k, y_params[i], save_path_HH)
    i += 1

In [ ]:
importlib.reload(sd)
# 3 layers
lmbd = 0.5
k = 2 * np.pi / lmbd 
r = np.array([0.125 * lmbd, 0.6 * lmbd, 1.0 * lmbd]) # concentric spheres radiuses
eps = np.array([1.0, 7.0, 5.0, 1.0] )
conducting_core = True

save_path_VV = f"../Theory/images/3_layers/3_layers_VV.png"
save_path_HH = f"../Theory/images/3_layers/3_layers_HH.png"

D_e, D_m = sd.calculate_coefficients(k, eps, r, conducting_core)
S_th, S_ph = sd.calculate_S(D_e, D_m)
ploting.plot_radar_cross_section(S_th, k, [-25, 25, 10], save_path_VV)
ploting.plot_radar_cross_section(S_ph, k, [-25, 25, 10], save_path_HH)